### Install dependencies

In [ ]:
%%bash
# git clone https://github.com/KaiyangZhou/Dassl.pytorch.git
cd Dassl.pytorch/
# Install dependencies
pip install -r requirements.txt
# Install this library (no need to re-build if the source code is modified)
python setup.py develop
cd ..
# Install dependencies of MaPLe
pip install -r requirements.txt

## generate ground truth instance labels

In [44]:
import os
import numpy as np
import torch
import glob

os.makedirs("../../dataset/data/replica_features/train", exist_ok=True)
os.makedirs("../../dataset/data/replica_features/val", exist_ok=True)
os.makedirs("../../dataset/data/replica_features/test", exist_ok=True)
train_test_dict = {
    "office0": "train",
    "office1": "train",
    "office2": "train",
    "office3": "val",
    "office4": "test",
    "room0": "train",
    "room1": "val",
    "room2": "test",
}

gt_masks = glob.glob("../../dataset/OpenYOLO3D/output/replica/replica_ground_truth_masks/*.pt")
# gt_instances = glob.glob("dataset/replica_split/ground_truth/*.txt")
for gt_mask_path in gt_masks:
    gt_mask = torch.load(gt_mask_path)[0]
    gt_instance_id_path = gt_mask_path.replace("OpenYOLO3D/output/replica/replica_ground_truth_masks/", "data/replica_split/ground_truth/").replace(".pt", ".txt")
    gt_instance_id = np.loadtxt(gt_instance_id_path, dtype='int')
    assert gt_mask.shape[0] == gt_instance_id.shape[0]
    
    labels = np.zeros(gt_mask.shape[1])
    for instance_id in range(gt_mask.shape[1]):
        mask_points = gt_mask[:, instance_id].bool()
        instance_label = gt_instance_id[mask_points][0]
        semantics_label = instance_label // 1000
        labels[instance_id] = semantics_label
    scene_name = os.path.splitext(os.path.basename(gt_mask_path))[0]
    split = train_test_dict[scene_name]
    output_path = f"../../dataset/data/replica_features/{split}/{scene_name}_labels"
    np.save(output_path, labels)

## generate instance feature (openscene features and ground truth masks)

In [60]:
# ADJUST PATH!!!
openscene_feature_path = glob.glob("../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/train/*_features.npy")
openscene_feature_path.extend(glob.glob("../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/val/*_features.npy"))
openscene_feature_path.extend(glob.glob("../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/test/*_features.npy"))

for feature_path in openscene_feature_path:
    print(feature_path)
    scene_name = os.path.splitext(os.path.basename(feature_path))[0]
    scene_name = scene_name.replace("_features", "")
    gt_mask_path = f"../../dataset/OpenYOLO3D/output/replica/replica_ground_truth_masks/{scene_name}.pt"

    point_features = np.load(feature_path)
    gt_mask = torch.load(gt_mask_path)[0]
    instance_features = np.zeros((gt_mask.shape[1], point_features.shape[1]))
    for instance_id in range(gt_mask.shape[1]):
        mask_points = gt_mask[:, instance_id].bool()
        mask_features = point_features[mask_points, :]
        instance_feature = np.mean(mask_features, axis=0)  # aggregate
        instance_features[instance_id, :] = instance_feature
    split = train_test_dict[scene_name]
    output_path = f"../../dataset/data/replica_features/{split}/{scene_name}_features"
    np.save(output_path, instance_features)
    

../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/train/office1_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/train/room0_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/train/office0_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/train/office2_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/val/room1_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/val/office3_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/test/office4_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/test/room2_features.npy


## Prompt Learning

In [1]:
# %%bash
# bash scripts/maple_prompt_scene/base2new_train_maple_prompt_scene.sh replica 1 ../../dataset/data/

Run this job and save the output to output/base2new/train_base/replica/shots_16/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx/seed1
Error while terminating subprocess (pid=2879446): 


In [3]:
# %%bash
# bash scripts/maple_prompt_scene/base2new_test_maple_prompt_scene.sh replica 1 ../../dataset/data/

Evaluating model
Runing the first phase job and save the output to output/base2new/test_new/replica/shots_16/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx/seed1


/cluster/54/nanriayaka/ml3d/models/multimodal-prompt-learning/clip/clip.py:23: UserWarning: PyTorch version 1.7.1 or higher is recommended
  warnings.warn("PyTorch version 1.7.1 or higher is recommended")


Error while terminating subprocess (pid=2879966): 


In [63]:
os.makedirs("../../dataset/data/replica_features/split_fewshot", exist_ok=True)

In [ ]:
"a {label} in a scene"

In [47]:
%%bash
bash scripts/maple_prompt_scene/xd_train_maple_prompt_scene.sh replica 1 ../../dataset/data/ vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets

Run this job and save the output to output/replica/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_adam_cross_datasets_1000000shots/seed1


/cluster/54/nanriayaka/ml3d/models/multimodal-prompt-learning/clip/clip.py:23: UserWarning: PyTorch version 1.7.1 or higher is recommended
  warnings.warn("PyTorch version 1.7.1 or higher is recommended")


Setting fixed seed: 1
***************
** Arguments **
***************
backbone: 
config_file: configs/trainers/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_adam_cross_datasets.yaml
dataset_config_file: configs/datasets/replica.yaml
eval_only: False
head: 
load_epoch: None
model_dir: 
no_train: False
opts: ['DATASET.NUM_SHOTS', '1000000']
output_dir: output/replica/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_adam_cross_datasets_1000000shots/seed1
resume: 
root: ../../dataset/data/
seed: 1
source_domains: None
target_domains: None
trainer: MaPLePromptScene
transforms: None
************
** Config **
************
DATALOADER:
  K_TRANSFORMS: 1
  NUM_WORKERS: 8
  RETURN_IMG0: False
  TEST:
    BATCH_SIZE: 100
    SAMPLER: SequentialSampler
  TRAIN_U:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAME_AS_X: True
    SAMPLER: RandomSampler
  TRAIN_X:
    BATCH_SIZE: 4
    N_DOMAIN: 0
    N_INS: 16
    SAMPLER: RandomSampler
DATASET:
  ALL_AS_UNLABELED: False
  CIFAR_C

val: [<datasets.replica.Datum_feature object at 0x7efcedc98400>, <datasets.replica.Datum_feature object at 0x7efcedc98460>, <datasets.replica.Datum_feature object at 0x7efcedc98d30>, <datasets.replica.Datum_feature object at 0x7efcedc98640>, <datasets.replica.Datum_feature object at 0x7efcedc986d0>, <datasets.replica.Datum_feature object at 0x7efcedc98580>, <datasets.replica.Datum_feature object at 0x7efcedc98700>, <datasets.replica.Datum_feature object at 0x7efcedc98af0>, <datasets.replica.Datum_feature object at 0x7efcedc98970>, <datasets.replica.Datum_feature object at 0x7efcedc988e0>, <datasets.replica.Datum_feature object at 0x7efcedc987f0>, <datasets.replica.Datum_feature object at 0x7efcedc98850>, <datasets.replica.Datum_feature object at 0x7efcedc988b0>, <datasets.replica.Datum_feature object at 0x7efcedc98a00>, <datasets.replica.Datum_feature object at 0x7efcedc98a60>, <datasets.replica.Datum_feature object at 0x7efcedc98ac0>, <datasets.replica.Datum_feature object at 0x7efced

-----------------------------------------------------------
SUBSAMPLE ALL CLASSES!
-----------------------------------------------------------
train: [<datasets.replica.Datum_feature object at 0x7efcedcc1720>, <datasets.replica.Datum_feature object at 0x7efcedcc1120>, <datasets.replica.Datum_feature object at 0x7efcedcc1270>, <datasets.replica.Datum_feature object at 0x7efcedcc08e0>, <datasets.replica.Datum_feature object at 0x7efcedcc14e0>, <datasets.replica.Datum_feature object at 0x7efcedcc0940>, <datasets.replica.Datum_feature object at 0x7efcedcc11e0>, <datasets.replica.Datum_feature object at 0x7efcedcc1300>, <datasets.replica.Datum_feature object at 0x7efcedcc13f0>, <datasets.replica.Datum_feature object at 0x7efcedcc13c0>, <datasets.replica.Datum_feature object at 0x7efcedcc1030>, <datasets.replica.Datum_feature object at 0x7efcedcc1960>, <datasets.replica.Datum_feature object at 0x7efcedcc1810>, <datasets.replica.Datum_feature object at 0x7efcedca8a00>, <datasets.replica.Datum

...

epoch [16/100] batch [20/44] time 0.037 (0.048) data 0.000 (0.009) loss 1.0146 (1.7023) lr 2.3810e-03 eta 0:02:59
epoch [16/100] batch [40/44] time 0.037 (0.043) data 0.000 (0.005) loss 2.4570 (1.8921) lr 2.3810e-03 eta 0:02:40
epoch [17/100] batch [20/44] time 0.037 (0.050) data 0.000 (0.010) loss 2.7266 (1.9319) lr 2.3638e-03 eta 0:03:04
epoch [17/100] batch [40/44] time 0.034 (0.044) data 0.000 (0.005) loss 1.2607 (1.8153) lr 2.3638e-03 eta 0:02:41
epoch [18/100] batch [20/44] time 0.039 (0.048) data 0.000 (0.009) loss 0.8730 (1.8315) lr 2.3454e-03 eta 0:02:55
epoch [18/100] batch [40/44] time 0.040 (0.044) data 0.000 (0.005) loss 2.7031 (1.7850) lr 2.3454e-03 eta 0:02:37
epoch [19/100] batch [20/44] time 0.039 (0.049) data 0.000 (0.009) loss 2.2148 (1.7526) lr 2.3259e-03 eta 0:02:56
epoch [19/100] batch [40/44] time 0.041 (0.045) data 0.000 (0.005) loss 2.2520 (1.7711) lr 2.3259e-03 eta 0:02:39
epoch [20/100] batch [20/44] time 0.039 (0.051) data 0.000 (0.011) loss 2.3594 (1.6519) 

epoch [52/100] batch [20/44] time 0.042 (0.050) data 0.000 (0.011) loss 0.6152 (0.9845) lr 1.2500e-03 eta 0:01:47
epoch [52/100] batch [40/44] time 0.040 (0.045) data 0.000 (0.006) loss 1.4521 (1.0982) lr 1.2500e-03 eta 0:01:35
epoch [53/100] batch [20/44] time 0.042 (0.049) data 0.000 (0.009) loss 1.2471 (0.9892) lr 1.2107e-03 eta 0:01:43
epoch [53/100] batch [40/44] time 0.039 (0.045) data 0.000 (0.004) loss 0.8379 (1.0716) lr 1.2107e-03 eta 0:01:32
epoch [54/100] batch [20/44] time 0.038 (0.049) data 0.000 (0.009) loss 0.6084 (0.8330) lr 1.1715e-03 eta 0:01:41
epoch [54/100] batch [40/44] time 0.041 (0.044) data 0.000 (0.005) loss 0.9941 (0.9939) lr 1.1715e-03 eta 0:01:29
epoch [55/100] batch [20/44] time 0.041 (0.048) data 0.000 (0.009) loss 0.5405 (0.9702) lr 1.1324e-03 eta 0:01:36
epoch [55/100] batch [40/44] time 0.037 (0.043) data 0.000 (0.005) loss 1.3379 (1.0582) lr 1.1324e-03 eta 0:01:24
epoch [56/100] batch [20/44] time 0.034 (0.046) data 0.000 (0.009) loss 1.2949 (0.9342) 

epoch [88/100] batch [20/44] time 0.041 (0.050) data 0.000 (0.009) loss 0.3799 (0.7983) lr 1.1897e-04 eta 0:00:27
epoch [88/100] batch [40/44] time 0.033 (0.045) data 0.000 (0.005) loss 0.7529 (0.7614) lr 1.1897e-04 eta 0:00:23
epoch [89/100] batch [20/44] time 0.041 (0.049) data 0.000 (0.010) loss 1.4092 (0.7116) lr 1.0281e-04 eta 0:00:24
epoch [89/100] batch [40/44] time 0.040 (0.044) data 0.000 (0.005) loss 1.5332 (0.7859) lr 1.0281e-04 eta 0:00:21
epoch [90/100] batch [20/44] time 0.039 (0.049) data 0.000 (0.009) loss 0.5957 (0.7681) lr 8.7779e-05 eta 0:00:22
epoch [90/100] batch [40/44] time 0.040 (0.044) data 0.000 (0.005) loss 1.1621 (0.7470) lr 8.7779e-05 eta 0:00:19
epoch [91/100] batch [20/44] time 0.040 (0.047) data 0.000 (0.009) loss 0.3735 (0.6921) lr 7.3899e-05 eta 0:00:19
epoch [91/100] batch [40/44] time 0.040 (0.044) data 0.000 (0.005) loss 0.4194 (0.7564) lr 7.3899e-05 eta 0:00:17
epoch [92/100] batch [20/44] time 0.041 (0.050) data 0.000 (0.009) loss 0.7700 (0.7355) 

  0%|          | 0/1 [00:00<?, ?it/s]/cluster/54/nanriayaka/ml3d/models/multimodal-prompt-learning/trainers/maple_prompt_scene.py:335: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print("softmax:", F.softmax(output))
100%|██████████| 1/1 [00:00<00:00,  3.56it/s]


output: tensor([[ -0.9453,  -6.7891,  -4.5664,  ...,  -3.2656,  -7.8906,  -3.1387],
        [  0.6489,  -6.6719,  -4.4922,  ...,  -3.0684,  -4.4102,  -0.5391],
        [ -2.1855,  -6.5547,  -8.3281,  ...,  -8.0469,  -4.4531,  -1.4258],
        ...,
        [-10.9609,  -9.2812, -14.6328,  ...,  -1.8369,   5.8711,  -9.1641],
        [ -8.1953,  -7.8828, -10.7500,  ...,   3.2559,   3.0977,  -8.3750],
        [  4.6992,  -6.2188,  -6.6016,  ..., -12.1250,  -4.8203,   7.4531]],
       device='cuda:0', dtype=torch.float16)
softmax: tensor([[5.1956e-03, 1.5080e-05, 1.3900e-04,  ..., 5.1022e-04, 5.0068e-06,
         5.7936e-04],
        [7.2250e-03, 4.7684e-06, 4.2260e-05,  ..., 1.7560e-04, 4.5896e-05,
         2.2030e-03],
        [9.3536e-03, 1.1837e-04, 2.0087e-05,  ..., 2.6643e-05, 9.6846e-04,
         1.9989e-02],
        ...,
        [0.0000e+00, 1.7881e-07, 0.0000e+00,  ..., 2.6464e-04, 5.8887e-01,
         1.7881e-07],
        [2.7418e-06, 3.7551e-06, 2.3842e-07,  ..., 2.5635e-01, 2.18

In [3]:
# !rm -r output/evaluation/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets_1000000shots/replica/seed1

In [4]:
%%bash
bash scripts/maple_prompt_scene/xd_test_maple_prompt_scene.sh replica 1 ../../dataset/data/ vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets

Run this job and save the output to output/evaluation/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets_1000000shots/replica/seed1


/cluster/54/nanriayaka/ml3d/models/multimodal-prompt-learning/clip/clip.py:23: UserWarning: PyTorch version 1.7.1 or higher is recommended
  warnings.warn("PyTorch version 1.7.1 or higher is recommended")


Setting fixed seed: 1
***************
** Arguments **
***************
backbone: 
config_file: configs/trainers/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets.yaml
dataset_config_file: configs/datasets/replica.yaml
eval_only: True
head: 
load_epoch: 2
model_dir: output/replica/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets_1000000shots/seed1
no_train: False
opts: []
output_dir: output/evaluation/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets_1000000shots/replica/seed1
resume: 
root: ../../dataset/data/
seed: 1
source_domains: None
target_domains: None
trainer: MaPLePromptScene
transforms: None
************
** Config **
************
DATALOADER:
  K_TRANSFORMS: 1
  NUM_WORKERS: 8
  RETURN_IMG0: False
  TEST:
    BATCH_SIZE: 100
    SAMPLER: SequentialSampler
  TRAIN_U:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAME_AS_X: True
    SAMPLER: RandomSampler
  TRAIN_X:
    BATCH_SIZE: 4
    N_DOMAIN: 0
    N_INS: 16
  

val: [<datasets.replica.Datum_feature object at 0x7f203523d4b0>, <datasets.replica.Datum_feature object at 0x7f203523d510>, <datasets.replica.Datum_feature object at 0x7f203523ddb0>, <datasets.replica.Datum_feature object at 0x7f203523d6f0>, <datasets.replica.Datum_feature object at 0x7f203523d780>, <datasets.replica.Datum_feature object at 0x7f203523d630>, <datasets.replica.Datum_feature object at 0x7f203523d7b0>, <datasets.replica.Datum_feature object at 0x7f203523db70>, <datasets.replica.Datum_feature object at 0x7f203523d9f0>, <datasets.replica.Datum_feature object at 0x7f203523d960>, <datasets.replica.Datum_feature object at 0x7f203523d870>, <datasets.replica.Datum_feature object at 0x7f203523d8d0>, <datasets.replica.Datum_feature object at 0x7f203523d930>, <datasets.replica.Datum_feature object at 0x7f203523da80>, <datasets.replica.Datum_feature object at 0x7f203523dae0>, <datasets.replica.Datum_feature object at 0x7f203523db40>, <datasets.replica.Datum_feature object at 0x7f2035

-----------------------------------------------------------
SUBSAMPLE ALL CLASSES!
-----------------------------------------------------------
train: [<datasets.replica.Datum_feature object at 0x7f203524a290>, <datasets.replica.Datum_feature object at 0x7f203524a2f0>, <datasets.replica.Datum_feature object at 0x7f203524a350>, <datasets.replica.Datum_feature object at 0x7f203524a3b0>, <datasets.replica.Datum_feature object at 0x7f203524a410>, <datasets.replica.Datum_feature object at 0x7f203524a470>, <datasets.replica.Datum_feature object at 0x7f203524a4d0>, <datasets.replica.Datum_feature object at 0x7f203524a530>, <datasets.replica.Datum_feature object at 0x7f203524a590>, <datasets.replica.Datum_feature object at 0x7f203524a5f0>, <datasets.replica.Datum_feature object at 0x7f203524a650>, <datasets.replica.Datum_feature object at 0x7f203524a6b0>, <datasets.replica.Datum_feature object at 0x7f203524a710>, <datasets.replica.Datum_feature object at 0x7f203524a770>, <datasets.replica.Datum

val: [<datasets.replica.Datum_feature object at 0x7f2035251c90>, <datasets.replica.Datum_feature object at 0x7f2035251cf0>, <datasets.replica.Datum_feature object at 0x7f2035251d50>, <datasets.replica.Datum_feature object at 0x7f2035251db0>, <datasets.replica.Datum_feature object at 0x7f2035251e10>, <datasets.replica.Datum_feature object at 0x7f2035251e70>, <datasets.replica.Datum_feature object at 0x7f2035251ed0>, <datasets.replica.Datum_feature object at 0x7f2035251f30>, <datasets.replica.Datum_feature object at 0x7f2035251f90>, <datasets.replica.Datum_feature object at 0x7f2035251ff0>, <datasets.replica.Datum_feature object at 0x7f2035252050>, <datasets.replica.Datum_feature object at 0x7f20352520b0>, <datasets.replica.Datum_feature object at 0x7f2035252110>, <datasets.replica.Datum_feature object at 0x7f2035252170>, <datasets.replica.Datum_feature object at 0x7f20352521d0>, <datasets.replica.Datum_feature object at 0x7f2035252230>, <datasets.replica.Datum_feature object at 0x7f2035

-----------------------------------------------------------
Label range: 0-47
Unique labels: {0, 3, 4, 5, 6, 7, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 30, 31, 32, 35, 36, 37, 38, 39, 40, 41, 43, 44, 45, 46, 47}
Label range: 0-47
Unique labels: {0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 16, 17, 19, 20, 21, 22, 24, 25, 26, 28, 29, 31, 35, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47}
Label range: 2-47
Unique labels: {2, 3, 5, 7, 8, 9, 10, 13, 14, 20, 21, 22, 25, 31, 33, 34, 37, 38, 41, 43, 44, 45, 46, 47}
Building transform_train
+ random resized crop (size=(224, 224), scale=(0.08, 1.0))
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
Building transform_test
+ resize the smaller edge to 224
+ 224x224 center crop
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
---------  -------
D

Traceback (most recent call last):
  File "/cluster/54/nanriayaka/ml3d/models/multimodal-prompt-learning/train.py", line 256, in <module>
    main(args)
  File "/cluster/54/nanriayaka/ml3d/models/multimodal-prompt-learning/train.py", line 194, in main
    trainer.load_model(args.model_dir, epoch=args.load_epoch)
  File "/cluster/54/nanriayaka/ml3d/models/multimodal-prompt-learning/trainers/maple_prompt_scene.py", line 371, in load_model
    raise FileNotFoundError('Model not found at "{}"'.format(model_path))
FileNotFoundError: Model not found at "output/replica/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets_1000000shots/seed1/MultiModalPromptSceneLearner/model.pth.tar-2"


CalledProcessError: Command 'b'bash scripts/maple_prompt_scene/xd_test_maple_prompt_scene.sh replica 1 ../../dataset/data/ vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets\n'' returned non-zero exit status 1.